In [ ]:
# !!! only run when using colab !!!
import os
! git clone https://github.com/Plant-Root-Soil-Interactions-Modelling/CPlantBox # downloading the source code
os.chdir("/content/CPlantBox/")
! cmake .
! make
os.chdir("/content/CPlantBox/tutorial/jupyter/workshop_11_2_2022/")

# Workshop 11.2.2022 - (1/3)

## CPlantBox basics

How to simulate a root system, understanding model parameters, use of confining geometry

### Importing the right modules

In [2]:
import sys; sys.path.append("../../.."); sys.path.append("../../../src/python_modules")
import plantbox as pb
import vtk_plot as vp # for quick vizualisations

### Simulate a root system

In [17]:
rs = pb.RootSystem()

# Open plant and root parameter from a file
path = "../../../modelparameter/rootsystem/"
name = "Anagallis_femina_Leitner_2010"
rs.readParameters(path + name + ".xml")
rs.initialize() 

rs.simulate(30) # [days]

# Export
rs.write("first_example.vtp") # for Paraview
rs.write("first_example.rsml") # e.g. gui/viewer/rsml_viewer.py

vp.plot_roots(rs, "creationTime") # Plot, using vtk (e.g. "subType")

(<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x57bec90) at 0x7ff077efdca0>,
 <vtkmodules.vtkRenderingAnnotation.vtkScalarBarActor(0x7ff04805a4c0) at 0x7ff077efdd00>)

Multiple root systems are created in the very same way, see *example2a_initializeparams.py*.

### Understanding model parameter

The seed contains the information to create the plant and is defined by the class *SeedRandomParameter*.
Each root type of emerging roots is represented by the class *RootRandomParameter*

Specific plants and roots (at their creation) are presented by the classes *SeedSpecificParameter* and *RootSpecificParameter*


In [3]:
random_seed = rs.getRootSystemParameter()
print(random_seed)

[Parameters of Anagallis_femina_Leitner_2010]
Variable		Value		Deviation		Description
===
maxTi             	0		0			Maximal number of tillers [1]
organType         	1		-			Organ type (unspecified organ = 0, seed = 1, root = 2, stem = 3, leaf = 4)
subType           	0		-			Unique identifier of this sub type
a                 	0.1		0			radius [cm]
delayB            	0		0			Time delay between the basal roots [day]
delayRC           	1000		0			Delay between the root crowns [day]
delaySB           	1000		0			Time delay between the shoot borne roots [day]
delayTi           	1e+09		0			Time delay between the tillers [day]
dx                	0.25		-			Axial resolution [cm] (maximal segment size)
dxMin             	1e-06		-			Axial resolution [cm] (minimal segment size)
firstB            	0		0			Emergence of first basal root [day]
firstSB           	1000		0			First emergence of a shoot borne root [day]
firstTi           	1e+09		0			Emergence of first tiller [day]
maxB              	0		0			Maxim

In [7]:
random_parameters = rs.getRootRandomParameter()
for p in random_parameters:
    print(p)

[Parameters of undefined]
Variable		Value		Deviation		Description
===
gf                	1		0			Growth function number [1]
organType         	2		-			Organ type (unspecified organ = 0, seed = 1, root = 2, stem = 3, leaf = 4)
subType           	0		-			Unique identifier of this sub type
tropismT          	1		-			Type of root tropism (plagio = 0, gravi = 1, exo = 2, hydro, chemo = 3)
a                 	0.1		0			radius [cm]
colorB            	0.2		-			Root color, blue component [0.-1.]
colorG            	0.2		-			Root color, green component [0.-1.]
colorR            	0.6		-			Root color, red component [0.-1.]
dx                	0.25		-			Axial resolution [cm] (maximal segment size)
dxMin             	1e-06		-			Axial resolution [cm] (minimal segment size)
la                	10		0			Apical zone [cm]
lb                	0		0			Basal zone [cm]
ldelay            	1		0			Lateral root emergence delay [day]
lmax              	0		0			Maximal root length [cm]
ln                	1		0			Inter-lateral d

All random parameters can be accessed and changed (especially usefull for dx, or standard deviations)

In [9]:
print("Apical length of root subType", random_parameters[1].subType,"is", random_parameters[1].la, "cm")
random_parameters[1].la = 10
for p in random_parameters:
    print("Maximal root length for subType", p.subType, "is", p.lmax, "cm", "sd", p.lmaxs)

Apical length of root subType 1 is 10.0 cm
Maximal root length for subType 0 is 0.0 cm sd 0.0
Maximal root length for subType 1 is 33.400002 cm sd 3.3
Maximal root length for subType 2 is 7.6999998 cm sd 10.0
Maximal root length for subType 3 is 0.44 cm sd 0.039999999
Maximal root length for subType 4 is 0.40000001 cm sd 0.039999999


Make the root system smaller

In [10]:
random_parameters[2].lmax = 1 # cm # subType 2 are the first order laterals
random_parameters[2].lmaxs = 0 # cm
rs.initialize() 
rs.simulate(30) 
vp.plot_roots(rs, "subType") # Plot, using vtk 

(<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x4717e40) at 0x7ff077efd640>,
 <vtkmodules.vtkRenderingAnnotation.vtkScalarBarActor(0x444eed0) at 0x7ff077efd4c0>)

The modified parameter set can be saved as xml parameter file. XML files can be conveniently viewed in a web browser. 

In [11]:
rs.writeParameters("modified_"+name + ".xml")

Specific plant and root parameters can be accessed in similar way, but they should not be changed (its a simulation result), and it is simpler to pull out such values using *getParameter*, which will be shown in the next jupyter notebook.

TODO parameters Figure for root and seed params

TODO widget for changing parameters

You do not need a parameter file, everything can be defined in the Python script, see *example2a_initializeparams*.

## Use of confining geometry or obstacles

Confining geometry and obstacles can be defined using signed distance functions (SDF), classes are defined in *src/sdf.h*. 

Its always a good idea to define the soil volume (even if unconstrained), because otherwise roots may grow upwards out of the soil (which is normally not to be expected).

In [12]:
rs = pb.RootSystem()
path = "../../../modelparameter/rootsystem/" # Open plant and root parameter from a file
name = "Zea_mays_4_Leitner_2014"
rs.readParameters(path + name + ".xml")
rs.setGeometry(pb.SDF_PlantBox(1.e6,1.e6,1.e6)) # length, width, height [-5.e5,5.e5] [-5.e5,5.e5] [0.,-1.e6]
rs.initialize()
rs.simulate(60)  # days
vp.plot_roots(rs, "creationTime") 

(<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x4463db0) at 0x7ff077efd940>,
 <vtkmodules.vtkRenderingAnnotation.vtkScalarBarActor(0x43e53f0) at 0x7ff077efda00>)

The following example is part of *example1c_complexcontainer.py* and describes a split box experiment

In [13]:
topBox = pb.SDF_PlantBox(22, 20, 5) # length, width, height [cm]
sideBox = pb.SDF_PlantBox(10, 20, 35)
left = pb.SDF_RotateTranslate(sideBox, pb.Vector3d(-6, 0, -5))
right = pb.SDF_RotateTranslate(sideBox, pb.Vector3d(6, 0, -5))
box_ = [topBox, left, right]
splitBox = pb.SDF_Union(box_)
print("distance to next boundary", topBox.getDist(pb.Vector3d(0, 0, -3)), "cm")

distance to next boundary -2.0 cm


In [15]:
rs.setGeometry(splitBox)
rs.initialize()
rs.simulate(60)  # days
rs.write("splitbox.vtp")
vp.plot_roots(rs, "creationTime") 

(<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x562c410) at 0x7ff077efd700>,
 <vtkmodules.vtkRenderingAnnotation.vtkScalarBarActor(0x452f770) at 0x7ff077efd8e0>)

More examples are given in *example1c_complexcontainer.py*. Visualization of the geometry can be achieved by writing a Paraview Python macro:

In [16]:
rs.write("splitbox.py") # can be run by Paraview